In [21]:
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

In [22]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)
X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_valid_wide, X_valid_deep = X_valid[:, :5], X_valid[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]

In [23]:
class WideAndDeepModel(tf.keras.Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.norm_layer_wide = tf.keras.layers.Normalization()
        self.norm_layer_deep = tf.keras.layers.Normalization()
        self.hidden1 = tf.keras.layers.Dense(units, activation=activation)
        self.hidden2 = tf.keras.layers.Dense(units, activation=activation)
        self.main_output = tf.keras.layers.Dense(1)
        self.aux_output = tf.keras.layers.Dense(1)

    def call(self, inputs):
        input_wide, input_deep = inputs
        norm_wide = self.norm_layer_wide(input_wide)
        norm_deep = self.norm_layer_deep(input_deep)
        hidden1 = self.hidden1(norm_deep)
        hidden2 = self.hidden2(hidden1)
        concat = tf.keras.layers.concatenate([norm_wide, hidden2])
        output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return output, aux_output

In [24]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
model = WideAndDeepModel(30, activation='relu', name='dynamic_model')

In [25]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss='mse', optimizer=optimizer, metrics=['RootMeanSquaredError'])
model.norm_layer_wide.adapt(X_train_wide)
model.norm_layer_deep.adapt(X_train_deep)
history = model.fit((X_train_wide, X_train_deep), y_train, epochs=20, validation_data=((X_valid_wide, X_valid_deep), y_valid))

Epoch 1/20
363/363 [==============================] - 2s 3ms/step - loss: 3.1215 - output_1_loss: 1.6629 - output_2_loss: 1.4586 - output_1_root_mean_squared_error: 1.2896 - output_2_root_mean_squared_error: 1.2077 - val_loss: 2.6315 - val_output_1_loss: 1.7551 - val_output_2_loss: 0.8765 - val_output_1_root_mean_squared_error: 1.3248 - val_output_2_root_mean_squared_error: 0.9362
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 1.1446 - output_1_loss: 0.5472 - output_2_loss: 0.5974 - output_1_root_mean_squared_error: 0.7397 - output_2_root_mean_squared_error: 0.7729 - val_loss: 1.5461 - val_output_1_loss: 0.6529 - val_output_2_loss: 0.8932 - val_output_1_root_mean_squared_error: 0.8080 - val_output_2_root_mean_squared_error: 0.9451
Epoch 3/20
363/363 [==============================] - 1s 3ms/step - loss: 1.0065 - output_1_loss: 0.4580 - output_2_loss: 0.5485 - output_1_root_mean_squared_error: 0.6768 - output_2_root_mean_squared_error: 0.7406 - val_loss: 1.473

In [26]:
eval_results_dict = model.evaluate((X_test_wide, X_test_deep), y_test, return_dict=True)
eval_results_dict

162/162 [==============================] - 0s 1ms/step - loss: 0.7147 - output_1_loss: 0.3296 - output_2_loss: 0.3850 - output_1_root_mean_squared_error: 0.5741 - output_2_root_mean_squared_error: 0.6205


{'loss': 0.7146704792976379,
 'output_1_loss': 0.3296428918838501,
 'output_2_loss': 0.38502761721611023,
 'output_1_root_mean_squared_error': 0.5741453766822815,
 'output_2_root_mean_squared_error': 0.6205059289932251}

Save/load model

In [27]:
model.save('test_models/dynamic_model', save_format='tf')

INFO:tensorflow:Assets written to: test_models/dynamic_model\assets


INFO:tensorflow:Assets written to: test_models/dynamic_model\assets


In [28]:
model = tf.keras.models.load_model('test_models/dynamic_model')
eval_results_dict = model.evaluate((X_test_wide, X_test_deep), y_test, return_dict=True)
eval_results_dict

162/162 [==============================] - 1s 1ms/step - loss: 0.7147 - output_1_loss: 0.3296 - output_2_loss: 0.3850 - output_1_root_mean_squared_error: 0.5741 - output_2_root_mean_squared_error: 0.6205


{'loss': 0.7146704792976379,
 'output_1_loss': 0.3296428918838501,
 'output_2_loss': 0.38502761721611023,
 'output_1_root_mean_squared_error': 0.5741453766822815,
 'output_2_root_mean_squared_error': 0.6205059289932251}

Callbacks - Checkpoints and early stopping

In [29]:
from pathlib import Path
from time import strftime

def get_run_logdir(root_logdir='my_logs'):
    return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")

run_logdir = get_run_logdir()

tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir, profile_batch=(100,200))

In [30]:
class PrintValTrainRatioCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        ratio = logs['val_loss'] / logs['loss']
        print(f"\nEpoch={epoch}, val./tr.={ratio:.2f}")

In [31]:
model_b = WideAndDeepModel(30, activation='relu', name='dynamic_model_b')
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model_b.compile(loss='mse', optimizer=optimizer, metrics=['RootMeanSquaredError'])
model_b.norm_layer_wide.adapt(X_train_wide)
model_b.norm_layer_deep.adapt(X_train_deep)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('test_checkpoints/model_b', save_weights_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
custom_cb = PrintValTrainRatioCallback()
history = model_b.fit((X_train_wide, X_train_deep), y_train, epochs=50, validation_data=((X_valid_wide, X_valid_deep), y_valid), 
                      callbacks=[checkpoint_cb, early_stopping_cb, custom_cb, tensorboard_cb])

Epoch 1/50
  1/363 [..............................] - ETA: 7:39 - loss: 15.8181 - output_1_loss: 8.6531 - output_2_loss: 7.1650 - output_1_root_mean_squared_error: 2.9416 - output_2_root_mean_squared_error: 2.6768WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0014s vs `on_train_batch_end` time: 0.0052s). Check your callbacks.


361/363 [============================>.] - ETA: 0s - loss: 3.6463 - output_1_loss: 1.9979 - output_2_loss: 1.6483 - output_1_root_mean_squared_error: 1.4135 - output_2_root_mean_squared_error: 1.2839
Epoch=0, val./tr.=0.54
363/363 [==============================] - 3s 4ms/step - loss: 3.6380 - output_1_loss: 1.9934 - output_2_loss: 1.6447 - output_1_root_mean_squared_error: 1.4119 - output_2_root_mean_squared_error: 1.2824 - val_loss: 1.9527 - val_output_1_loss: 1.0484 - val_output_2_loss: 0.9043 - val_output_1_root_mean_squared_error: 1.0239 - val_output_2_root_mean_squared_error: 0.9509
Epoch 2/50
350/363 [===========================>..] - ETA: 0s - loss: 1.3308 - output_1_loss: 0.6726 - output_2_loss: 0.6582 - output_1_root_mean_squared_error: 0.8201 - output_2_root_mean_squared_error: 0.8113
Epoch=1, val./tr.=0.97
363/363 [==============================] - 1s 3ms/step - loss: 1.3269 - output_1_loss: 0.6694 - output_2_loss: 0.6574 - output_1_root_mean_squared_error: 0.8182 - output_